**<h1><b>Applied Machine Learning<b></h1>**
<h2>Αλέξανδρος - Ιωάννης Δουνάκης</h2> 
<h3>3170044</h3>




## <h3>Εργασία 1 </h5>

<h3><b>Q1</b></h6>
Obtaining the Data 



In [333]:
import numpy as np
import pandas as pd
import matplotlib as plt

raw_data = pd.read_csv("2757408.csv",parse_dates=['DATE'] )
#raw_data.DATE = pd.to_datetime(raw_data.DATE)

raw_data2 = pd.read_csv("athens.csv",parse_dates=['DATE'])
#raw_data2.DATE = pd.to_datetime(raw_data2.DATE)

<h3>Αρχή Διαχείρισης 1ου Σετ Δεδομένων </h3>
Διασχίζουμε τα dataset προκειμένου να δούμε ποια δεδομένα λείπουν.
Βλέπουμε πως οι NaN τιμές ειναι πολλές στο column TAVG , η οποία σίγουρα θα χρειαστεί προκειμένου να κάνουμε την ανάλυση

In [334]:
print("RAW_DATA NAN VALUES IN TAVG \n",raw_data.TAVG.isna().sum())
print("RAW_DATA NAN VALUES IN TMAX \n",raw_data.TMAX.isna().sum())
print("RAW_DATA NAN VALUES IN TMIN \n",raw_data.TMIN.isna().sum())
print("RAW_DATA NAN VALUES IN PRCP \n",raw_data.PRCP.isna().sum())

RAW_DATA NAN VALUES IN TAVG 
 2311
RAW_DATA NAN VALUES IN TMAX 
 944
RAW_DATA NAN VALUES IN TMIN 
 833
RAW_DATA NAN VALUES IN PRCP 
 478


Θέλοντας να φτιάξουμε τα δεδομένα χρειάζεται να αλλάξουμε τις τιμές NaN σε μηδέν έτσι ώστε να είναι πιο διαχειρίσιμες

In [335]:
raw_data.TAVG.fillna(0,inplace=True)
raw_data.TMAX.fillna(0,inplace=True)    
raw_data.TMIN.fillna(0,inplace=True)
raw_data.PRCP.fillna(0,inplace=True)


Συνεπώς , πρεπει να διαμορφώσουμε τα δεδομένα τα οποία λείπουν απο το 1ο σετ δεδομένων.<br>
Αρχικά τα δεδομένα που χρειάζονται διαμόρφωση είναι όσες τιμές του TAVG == 0 εφόσον νωρίτερα τρέξαμε την <br><code>raw_data.TAVG.fillna(0,inplace=True)</code> 
<br>
Συνεπώς τα βήματα που ακολουθούμε ειναι τα εξής:
<br>
<lu>
    1. Κατατόπιση των: <br>
        <li>A: Το σύνολο των γραμμών για τις οποίες ισχύει TAVG =0 και ΤΜΑΧ != 0 και ΤΜΙΝ != 0 </li>
        <li>Β: Το σύνολο των γραμμών για τις οποίες ισχύει TAVG =0 και ΤΜΑΧ =0 ή ΤΜΙΝ =0 </li>
        <br>
    2. Για το σύνολο Α , θέτουμε την τιμη TAVG όσο το AVG των στοιχείων TMAX και TMIN (στη περίπτωση μας το AVG ταυτίζεται με το median) <br>
    3. Για το σύνολο Β , θέτουμε την τιμη TAVG ίσο με την τιμη TMAX / TMIN ή οποία ειναι διαφορετική απο μηδέν.
</lu>
<br>
THIS WORKS
we dont take into consideration the case where (TAVG > TMAX) | (TAVG < TMIN)

In [336]:
raw_data.TAVG.mask( raw_data.TAVG == '---' , 0 , inplace=True)
raw_data.TAVG.mask((raw_data.TAVG == 0) & (raw_data.TMAX != 0) & (raw_data.TMAX != 0) , (raw_data.TMAX +raw_data.TMIN) /2 ,inplace =True)
raw_data.TAVG.mask((raw_data.TAVG == 0) & ((raw_data.TMAX == 0) | (raw_data.TMIN == 0)),raw_data.TMAX + raw_data.TMIN, inplace = True)


Παρατηρώντας τα δεδομένα του αρχικού σετ δεδομένων , παρατηρούμε πως η μονάδα μέτρησης θερμοκρασίας δεν ειναι κελσίου , οπότε κάνουμε Fahrenheit to Celcius.

In [337]:
raw_data.TAVG.mask(raw_data.TAVG > 0 ,( (raw_data.TAVG - 32.0) * 5.0 / 9.0) , inplace=True)

Επίσης για την διαμόρφωση των δεδομένων χρειάζεται να ελέγξουμε την πληρότητα των τιμών στο πεδίο <code>raw_data.PRCP</code> <br>
Εξετάζοντας τα δεδομένα αυτά βλέπουμε πως το ποσοστο των τιμών NaN για τα έτη 2010 μέχρι 2019 είναι πολύ μεγάλο έτσι ώστε να μπορούμε να εξάγουμε συμπεράσματα. <br>
Συνεπώς θα χρειαστούμε να αντικαταστήσουμε τα δεδομένα που λείπουν απο το 2ο dataset που μας δίνεται , τρέχουμε την εντολή αυτή αργότερα σε διαφορετικό κελί. <br>


In [338]:
tocount = raw_data.PRCP.loc[(raw_data.DATE > '2010-1-1') & (raw_data.DATE < '2020-1-1') ]
print("PERCENTAGE OF NAN IN PRCP THROUGH 2010 - 2019\n",((tocount == 0).sum())/tocount.size *100)

PERCENTAGE OF NAN IN PRCP THROUGH 2010 - 2019
 86.24017155110793


Τέλος , θέλουμε να δούμε την πληρότητα στις μέρες που καλύπτει το αρχικό dataset.
Βάση του παρακάτω κώδικα βλέπουμε πως απο τις 365* (2020-2010) ~ 3.652 μέρες που υπάρχουν σε 10 χρόνια , στο σετ εμφανίζονται πολύ λιγότερες.<br>
Συνεπώς θα χρειαστούμε να πάρουμε τα δεδομένα για τις μέρες οι οποίες λείπουν ,απο το 2ο σετ.<br>
Βλέπουμε πως όντως το 2ο σετ δεδομένων ειναι πολυ πιο πλήρες για την 10ετία 2010 με 2019.

In [339]:
days = raw_data.DATE.loc[(raw_data.DATE >='2010-1-1') & (raw_data.DATE < '2020-1-1')]
days2 = raw_data2.DATE.loc[(raw_data2.DATE >= '2010-1-1') & (raw_data2.DATE < '2020-1-1')]

print("PERCENTAGE OF DAYS \n",days.size / days2.size *100)

PERCENTAGE OF DAYS 
 76.64293537787513


<h3>Αρχη διαχειρισης 2ο Σετ Δεδομένων</h3>
Ωστόσο στο 2o Dataset δεν φαίνεται να λείπουν δεδομένα χρήσιμα σε εμάς.

In [340]:
print("RAW_DATA2 NAN VALUES IN TAVG \n",raw_data2.TAVG.isna().sum())
print("RAW_DATA2 NAN VALUES IN TMAX \n",raw_data2.TMAX.isna().sum())
print("RAW_DATA2 NAN VALUES IN TMIN \n",raw_data2.TMIN.isna().sum())
print("RAW_DATA2 NAN VALUES IN PRCP \n",raw_data2.PRCP.isna().sum())

# (raw_data.TAVG == float(0)).sum()

RAW_DATA2 NAN VALUES IN TAVG 
 0
RAW_DATA2 NAN VALUES IN TMAX 
 0
RAW_DATA2 NAN VALUES IN TMIN 
 0
RAW_DATA2 NAN VALUES IN PRCP 
 0


Raw_data2.PRCP normalization to [0.0-1.0] ,as seen in original raw_data  

In [341]:
raw_data2.PRCP.mask( raw_data2.PRCP >0 , raw_data2.PRCP/100 , inplace=True)
#df.drop(df[df.score < 50].index, inplace=True)
raw_data2.drop(raw_data2[raw_data2.TAVG == '---'].index , inplace=True)
raw_data2[raw_data2.TAVG =='---']


,DATE,TAVG,TMAX,TMIN,HUMIDITY,PRCP,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23


Αφού τροποποιήσαμε τα δεδομένα μας και στα 2 σετ που δίνονται , τώρα είναι έτοιμα ετσι ώστε να κάνουμε συνένωση σε έναν ενιαίο πίνακα.<br>
Για την σωστή συνένωση , βρίσκουμε ποιές ημερομηνίες του 2ου σετ υπάρχουν στο 1ο (που συνεπώς δεν τις χρειαζόμαστε) και τις αφαιρούμε. <br>
Την ένωση των δεδομένων την πραγματοποιούμε με τη μέθοδο <code>pd.concat([dataset1 , dataset2])</code> , ακόμα πετάμε τα column που δεν μας χρησιμεύουν.<br>
Παρακάτω εμφανίζουμε τον τελικό πίνακα <code>data</code> ταξινομημένο βάση του <code>index</code> (DATE) ώντας ταξινομημένα απο μικρότερο προς μεγαλύτερο.

In [342]:
raw_data2 = raw_data2[~raw_data2.DATE.isin(raw_data.DATE)]
data = raw_data.append(raw_data2).copy()
data = data[['DATE' ,'TAVG' , 'PRCP']]
data


,DATE,TAVG,PRCP
0,1955-01-01,14.166667,0.00
1,1955-01-02,10.833333,0.08
2,1955-01-03,12.5,0.00
3,1955-01-04,13.055556,0.00
4,1955-01-05,13.055556,0.00
...,...,...,...
2831,2017-10-02,18.7,0.77
2837,2017-10-08,17.8,0.64
2844,2017-10-15,20.4,0.65
2856,2017-10-27,17.8,0.81


<h3> Δημιουργία DataFrames για την αποθήκευση των στατιστικών</h3> <br>


In [343]:
month_avg = pd.DataFrame({'YEAR': data.DATE.dt.year,'MONTH': data.DATE.dt.month,'TAVG': data.TAVG , 'PRCP': data.PRCP})
#print(month_avg.YEAR.sort_values())

month_avg = month_avg.astype({"TAVG" : float})
month_avg = month_avg.groupby(['YEAR','MONTH']).mean('TAVG').sort_values(by=['YEAR','MONTH'])
month_avg
# month_avg.head(12)
# month_avg.TAVG = data.mean()
# month_avg.YEAR.sort_values()
# monthlyAvg = monthlyAvg.groupby(['YEAR','MONTH'])
# monthlyAvg.head(12)


TAVG      PRCP
YEAR MONTH                     
1955 1      13.118280  0.085484
     2      13.859127  0.046071
     3      12.777778  0.022258
     4      14.212963  0.045000
     5      20.277778  0.000323
...               ...       ...
2020 8      28.781362  0.018065
     9      26.092593  0.003000
     10     21.218638  0.010645
     11     15.481481  0.006667
     12     14.265233  0.026774

[792 rows x 2 columns]

YEARLY AVERAGE 

In [344]:
year_avg = pd.DataFrame({'YEAR' : data.DATE.dt.year , 'TAVG' : data.TAVG , 'PRCP' : data.PRCP})
year_avg = year_avg.astype({"TAVG" :float})
year_avg = year_avg.groupby('YEAR').mean('TAVG').sort_values(by='YEAR')
year_avg

,TAVG,PRCP
YEAR,,
1955,18.779300,0.059041
1956,18.153461,0.030273
1957,18.356925,0.032630
1958,18.694064,0.037781
1959,17.919330,0.023068
...,...,...
2016,19.375440,0.028388
2017,19.175221,0.075671
2018,19.504323,0.042603
